In [14]:
import pandas as pd
import re
import seaborn as sns
import os
import matplotlib as plt
import statsmodels.api as sm

#### Split the dataset into 2 parts, 60% of it will be used to create a dataset for winner, the rest is for loser, the 2 datasets then will be merged again to create a dataset with a y variable


In [15]:
df = pd.read_csv(r"C:\Users\Đức\OneDrive - CVA\HUU DUC onedrive\Research paper with Dr.Oleg\dataset\df_match.csv")

df = df.sample(frac = 1, random_state=5).reset_index(drop=True)

n = int(len(df)*0.6)
winner_df = df.iloc[:n]
loser_df = df.iloc[n:]



In [16]:
winner_df = winner_df.rename(columns={'winner_id': 'player_id',
                        'winner_seed': 'player_seed',
                        'winner_name': 'player_name',
                        'winner_hand': 'player_hand',
                        'winner_ht': 'player_ht',
                        'winner_ioc': 'player_ioc',
                        'winner_age': 'player_age',
                        'winner_entry': 'player_entry',
                        'winner_rank': 'player_rank',
                        'winner_rank_points': 'player_rank_points'})

# Add a column to represent difference in seed (there are so many missing values)
winner_df['seed_dif'] = winner_df['player_seed'] - winner_df['loser_seed']

# Difference in height
winner_df['ht_dif'] = winner_df['player_ht'] - winner_df['loser_ht']

# Difference in age
winner_df['age_dif'] = winner_df['player_age'] - winner_df['loser_age']

# Difference in rank and rank points
winner_df['rank_dif'] = winner_df['player_rank'] - winner_df['loser_rank']
winner_df['rank_points_dif'] = winner_df['player_rank_points'] - winner_df['loser_rank_points']

winner_df = winner_df[['player_id',
                        'player_seed', 'seed_dif',
                        'player_name',
                        'player_hand',
                        'player_ht', 'ht_dif',
                        'player_ioc',
                        'player_age', 'age_dif',
                        'player_entry',
                        'player_rank', 'rank_dif',
                        'player_rank_points', 'rank_points_dif',
                        'tourney_name',
                        'surface',
                        'draw_size',
                        'tourney_level',
                        'tourney_date',
                        'round']]

winner_df['outcome'] = 1

In [17]:
loser_df = loser_df.rename(columns={'loser_id': 'player_id',
                        'loser_seed': 'player_seed',
                        'loser_name': 'player_name',
                        'loser_hand': 'player_hand',
                        'loser_ht': 'player_ht',
                        'loser_ioc': 'player_ioc',
                        'loser_age': 'player_age',
                        'loser_entry': 'player_entry',
                        'loser_rank': 'player_rank',
                        'loser_rank_points': 'player_rank_points'})


# Add a column to represent difference in seed (there are so many missing values)
loser_df['seed_dif'] = loser_df['player_seed'] - loser_df['winner_seed']

# Difference in height
loser_df['ht_dif'] = loser_df['player_ht'] - loser_df['winner_ht']

# Difference in age
loser_df['age_dif'] = loser_df['player_age'] - loser_df['winner_age']

# Difference in rank and rank points
loser_df['rank_dif'] = loser_df['player_rank'] - loser_df['winner_rank']
loser_df['rank_points_dif'] = loser_df['player_rank_points'] - loser_df['winner_rank_points']

loser_df = loser_df[['player_id',
                        'player_seed', 'seed_dif',
                        'player_name',
                        'player_hand',
                        'player_ht', 'ht_dif',
                        'player_ioc',
                        'player_age', 'age_dif',
                        'player_entry',
                        'player_rank', 'rank_dif',
                        'player_rank_points', 'rank_points_dif',
                        'tourney_name',
                        'surface',
                        'draw_size',
                        'tourney_level',
                        'tourney_date',
                        'round']]

loser_df['outcome'] = 0

In [18]:
model_df = pd.concat([winner_df, loser_df], axis = 0, ignore_index=True)
model_df = model_df.sample(frac = 1, random_state = 10).reset_index(drop = True)

model_df = pd.get_dummies(data = model_df, columns =['surface'], drop_first=True)

model_df = model_df.dropna()  

col_list = ['surface_clay', 'surface_grass', 'surface_hard', 'surface_unknown']
for col in col_list:
    model_df[col] = model_df[col].apply(lambda x: 0 if x==False else 1)

train_size = int(len(model_df)*0.7)
train_df = model_df.iloc[:train_size]
test_df = model_df.iloc[train_size:]



In [20]:
X_train = train_df[['ht_dif', 'age_dif', 'rank_dif', 'rank_points_dif','surface_clay', 'surface_grass', 'surface_hard']]
y_train = train_df['outcome']

X_train = sm.add_constant(X_train)
model = sm.Logit(y_train, X_train)
result = model.fit()

print(result.summary())


         Current function value: 0.647057
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                outcome   No. Observations:                  504
Model:                          Logit   Df Residuals:                      496
Method:                           MLE   Df Model:                            7
Date:                Wed, 06 Aug 2025   Pseudo R-squ.:                 0.02341
Time:                        12:55:12   Log-Likelihood:                -326.12
converged:                      False   LL-Null:                       -333.93
Covariance Type:            nonrobust   LLR p-value:                   0.02867
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.2766      0.546      0.507      0.612      -0.793       1.346
ht_dif             -0.0079      0.008     -0.974  

c:\Users\Đức\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
